# Quick and dirty RAG implementation using LlamaIndex and OpenAI API

# Install packages

In [ ]:
!pip install llama-index

In [7]:
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings

Settings.llm = OpenAI(model="gpt-4o")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")
Settings.node_parser = SentenceSplitter(chunk_size=512, chunk_overlap=20)
Settings.num_output = 512
Settings.context_window = 3900

# Define documents used for RAG and create an index

In [31]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
documents = SimpleDirectoryReader(input_dir="data/input", recursive=True).load_data()
index = VectorStoreIndex.from_documents(documents) # Not being stored to a vectorDB
query_engine = index.as_query_engine()

# Use the query engine to get answers based on provided documents

In [ ]:
RAG_response = query_engine.query("Name some user research methodologies")
RAG_response

In [ ]:
RAG_response2 = query_engine.query("What do you know about Alex Khomutov?")
RAG_response2

In [ ]:
RAG_response3 = query_engine.query("Has he worked on any automotive projects?")
RAG_response3


## Create a Discord Bot to talk to the RAG

In [39]:
!pip install discord.py asyncio
!pip install audioop-lts


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


### Import packages

In [41]:
import asyncio
import discord
import dotenv
import os

In [42]:

dotenv.load_dotenv()
DISCORD_TOKEN = os.getenv("DISCORD_TOKEN")
intents = discord.Intents.default()
intents.message_content = True  # Required to read messages
client = discord.Client(intents=intents)

@client.event
async def on_ready():
    print(f'We have logged in as {client.user}')

@client.event
async def on_message(message):
    if message.author == client.user:
        return  # Ignore messages from the bot itself

    if message.content.startswith("!rag"):  # Or any other prefix you want to use
        query = message.content[5:]  # Remove the "!rag" prefix
        print(f"Received query: {query}")

        try:
            RAG_response = query_engine.query(query)
            await message.channel.send(str(RAG_response))  # Send the response back to Discord

        except Exception as e:
            await message.channel.send(f"An error occurred: {e}")
            print(f"Error during query processing: {e}")

async def main():
    await client.start(DISCORD_TOKEN) 

# Run the bot in a separate task, so the notebook doesn't block
asyncio.create_task(main())

RuntimeError: asyncio.run() cannot be called from a running event loop